# Decorators

A decorator is just a function that takes in another function as an argument, adds some functionality to it (without modifying the actual function) and returns it with the added functionality.

For example, we'll create a decorator, called `timer`, that just times the execution of a function.

In [68]:
from time import time

In [93]:
# note the decorator only takes 
# in func as a parameter.
def timer(func):
    def wrapper():
        start = time()
        result = func()
        end = time()
        print(f'{func.__name__} took {end - start:.4f} seconds to run.')
        return result
    return wrapper

`timer` takes in a function and returns a new function that just times how long it takes our function to execute and prints the results. The function the decorator returns is commonly called wrapper because it *wraps* our original function with extra functionaity. 

We'll define a function that takes some time to compute, then we can *decorate* it with our timer decorator to see how long it takes.

In [82]:
# waste_time just sums up all the numbers 
# between 1 and 10 million.
def waste_time():
    sum = 1
    for i in range(1, 10_000_001):
        sum += i  
    return sum

print('run waste_time once before we decorate it')
print(f'{waste_time():,}')

# To decorate this function with our timer we can just write:
waste_time = timer(waste_time)

run waste_time once before we decorate it
50,000,005,000,001


In [83]:
waste_time()

waste_time took 0.6890 seconds to run.


50000005000001

We see that in order to apply our decorator we have to call
```python
waste_time = timer(waste_time)
```
or more generally
```python
func = decorator(func)
```
but python also offers a different way of applying a decorator to a function, just add
```python 
@decorator
def func():
    ...
```
This method of decorating a function is much more common than the former.

In [84]:
@timer
def waste_time():
    sum = 1
    for i in range(1, 10_000_001):
        sum += i  
    return sum

In [85]:
waste_time()

waste_time took 0.6864 seconds to run.


50000005000001

```python
@decorator
def func():
    ...
```
is just [syntactic sugar](https://en.wikipedia.org/wiki/Syntactic_sugar) for 
```python
func = decorator(func)
```

## Decorators for functions with parameters

But so far we have a problem. What if our function takes in parameters? \
In this case we just have to add the \*args and \**kwargs to our wrapper parameters and function parameters. This way we ensure that any parameters our function may take get passed along to it. This will still work for functions that take no arguments (both args and kwargs will be empty)

As a basic example, we'll redefine our waste_time function to take in a parameter, n, that determines what number to iterate up to.
We also need to modify our timer decorator in order to pass along the argumnent to waste_time in the wrapper.

In [89]:
def timer(func):
    def wrapper(*args, **kwargs):
        start = time()
        result = func(*args, **kwargs)
        end = time()
        print(f'{func.__name__} took {end - start:.4f} seconds to run.')
        return result
    return wrapper

In [90]:
@timer
def waste_time(n=10_000_000):
    sum = 0
    for i in range(1, n + 1):
        sum += i
    return sum

In [91]:
waste_time(50_000_000)

waste_time took 3.6096 seconds to run.


1250000025000000

## Decorators with arguments

We can even create decorators that take in arguments when they decorate a function. For example let's say we have some sort of mathematical function $f(x)$. If we wanted to compose it on itself, we would just do $f(f(x))$ or for ease of notation we'll call it $f^2(x)$. If we wanted to compose $f$ on itself 3 times we would just do $f(f(f(x)))$ or just $f^3(x)$.

Let's write a decorator that let's us compose a function on itself n times.

In order to do this we just take our existing structure of defining a decorator and just wrap it all in just one more function that would take in this new parameter.

In [95]:
def compose(n=1):
    def compose_dec(func):
        def wrapper(*args, **kwargs):
            res = func(*args, **kwargs)
            for i in range(n-1):
                res = func(res)
            return res
        return wrapper
    return compose_dec

We created our decorator, now we'll create a simple function that takes in a number, $x$, and returns $x + 1$.

In [107]:
@compose(3)
def plus_one(x):
    return x + 1

In [108]:
plus_one(10)

13

Since we composed plus_one on itself 3 times, we effecitvely just get a new function that adds 3 to any number.

In [125]:
@compose(2)
def concat(string):
    return string + ' ' +  string

In [126]:
concat('hello')

'hello hello hello hello'